In [ ]:
!pip install ../input/pretrainedmodels-new2/pretrainedmodels-0.7.4/ > /dev/null
!pip install ../input/efficientnet-pytorch/ > /dev/null
!pip install ../input/timm-pytorch-image-models/pytorch-image-models-master/ > /dev/null
!pip install ../input/segmodelqubvel/segmentation_models.pytorch-master/ > /dev/null


In [ ]:
package_path = '../input/segmodelqubvel/segmentation_models.pytorch-master/segmentation_models_pytorch' # add unet script dataset
import sys
sys.path.append(package_path)
from segmentation_models_pytorch import Unet, FPN, DeepLabV3Plus

In [ ]:
import pdb
import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset
from albumentations import (Normalize, Compose)
from albumentations.pytorch import ToTensor
import torch.utils.data as data
#from model import Unet
import glob

In [ ]:
models=[]

In [ ]:
## Model with best score

ckpt_path = "../input/models-steel-1/unet_seresnet50_bce.pth"
device = torch.device("cuda")
model = Unet("se_resnet50", encoder_weights=None, classes=4, activation=None)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
models.append(model)

In [ ]:
## Model with 2nd best score

ckpt_path = "../input/models-steel-1/fpn_efficientnetb3_bce.pth"
device = torch.device("cuda")
model = FPN("efficientnet-b3", encoder_weights=None, classes=4, activation=None)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
models.append(model)

In [ ]:
## Model with 3rd best score

ckpt_path = "../input/models-steel-1/fpn_efficientnet_b1.pth"
device = torch.device("cuda")
model = FPN("efficientnet-b1", encoder_weights=None, classes=4, activation=None)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
models.append(model)

In [ ]:
## Model with 4th best score
'''
ckpt_path = "../input/models-steel-1/deeplabv3plus_resnet34_bce.pth"
device = torch.device("cuda")
model = DeepLabV3Plus("resnet34", encoder_weights=None, classes=4, activation=None)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
models.append(model)'''

In [ ]:
## Model with 5th best score

ckpt_path = "../input/models-steel-1/fpn_resnet34_bce.pth"
device = torch.device("cuda")
model = FPN("resnet34", encoder_weights=None, classes=4, activation=None)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
models.append(model)


In [ ]:
## Model with 6th best score
'''
ckpt_path = "../input/models-steel-1/unet_se_resnext_bce.pth"
device = torch.device("cuda")
model = Unet("se_resnext50_32x4d", encoder_weights=None, classes=4, activation=None)
model.to(device)
model.eval()
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
models.append(model)
'''

In [ ]:
test_image_names=[]
for file in glob.glob("../input/severstal-steel-defect-detection/test_images/*.jpg"):
    test_image_names.append(file.split('/')[-1])
sorted(test_image_names)
#test_image_names=test_image_names[:8]
print(test_image_names[:5])
print(len(test_image_names))

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, df, mean, std,test_idx):
        self.root = root
        #df['ImageId'] = df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
        #self.fnames = df['ImageId'].unique().tolist()
        self.fnames=test_idx
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Normalize(mean=mean, std=std, p=1),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return fname, images

    def __len__(self):
        return self.num_samples

In [ ]:
def post_process(probability, threshold, min_size):
    '''Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored'''
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((256, 1600), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [ ]:
sample_submission_path = '../input/severstal-steel-defect-detection/sample_submission.csv'
test_data_folder = "../input/severstal-steel-defect-detection/test_images"

In [ ]:
best_threshold = [0.5,0.6,0.45,0.4]
num_workers = 2
batch_size = 1
print('best_threshold', best_threshold)
min_size = [1400,1800,1200,1300]
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
df = pd.read_csv(sample_submission_path)
testset = DataLoader(
    TestDataset(test_data_folder, df, mean, std,test_image_names),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

In [ ]:
predictions = []
for i, batch in enumerate(tqdm(testset)):
    fnames, images = batch
    images=images.to(device)
    batch_preds=torch.zeros((len(fnames),4,256,1600)).to(device)
    num_aug=0
    for i in range(len(models)):
        if 1:
            batch_preds += torch.sigmoid(models[i](images))
            num_aug += 1
        if 1:
            batch_preds += torch.flip(torch.sigmoid(models[i](torch.flip(images,dims=[2]))),dims=[2])
            num_aug += 1
        if 1:
            batch_preds += torch.flip(torch.sigmoid(models[i](torch.flip(images,dims=[3]))),dims=[3])
            num_aug += 1
    
    batch_preds = batch_preds.detach().cpu().numpy()/(num_aug)
    #print(num_aug)
    for fname, preds in zip(fnames, batch_preds):
        for cls, pred in enumerate(preds):
            pred, num = post_process(pred, best_threshold[cls], min_size[cls])
            rle = mask2rle(pred)
            name = fname + f"_{cls+1}"
            predictions.append([name, rle])

# save predictions to submission.csv
df = pd.DataFrame(predictions, columns=['ImageId_ClassId', 'EncodedPixels'])
df.to_csv("submission.csv", index=False)

In [ ]:
num_aug